In [1]:
%load_ext autoreload
%autoreload 2
from data_translator import CMAToScheduleTranslator, TestTranslator, EncodeForCMA, EncodeForGA, GAToScheduleTranslator
from hybrid_solution_data_loader import get_data
from models import Order, SimulationEnvironment, Schedule
from solver import CMAWeightSolver, HybridGreedyAgentSolver
from visualize import visualize_schedule_demo
import objective_function

In [2]:
n_workstations, recipes, operation_times = get_data(3)
recipies, workstations, resources, tasks, _ = TestTranslator().translate(n_workstations, recipes, operation_times)

env = SimulationEnvironment(workstations, tasks, resources, recipies)

In [3]:
earliest_slot = 0
last_slot = 100
recipe_orders = [0, 1, 2, 3, 0, 3, 2, 1, 0, 3] # for dataset 0
orders = []
o_id = 0
for order in recipe_orders:
    orders.append(Order(o_id, 0, last_slot, last_slot, [order], 100, 50, False, 0, False, 500)) # for now: use resources to select recipe
    o_id = o_id + 1

In [4]:
encoder : EncodeForCMA = EncodeForCMA()
values, durations, jobs, due_dates = encoder.translate(env, orders)

In [5]:
solver : CMAWeightSolver = CMAWeightSolver(values, durations, jobs, env, orders)
solver.initialize() # doesn't do anything right now
solver.run()

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 40 (seed=896675, Mon Jun 26 14:36:05 2023)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 2.000000000000000e+00 1.0e+00 4.70e-01  5e-01  5e-01 0:02.5
    2     30 2.000000000000000e+00 1.0e+00 4.49e-01  4e-01  5e-01 0:05.0
    3     45 2.000000000000000e+00 1.1e+00 4.37e-01  4e-01  4e-01 0:07.5
    5     75 2.000000000000000e+00 1.1e+00 4.26e-01  4e-01  4e-01 0:12.5
    7    105 7.900000000000000e+01 1.1e+00 4.44e-01  4e-01  5e-01 0:17.6
   10    150 2.000000000000000e+00 1.2e+00 4.67e-01  5e-01  5e-01 0:24.5
   13    195 2.000000000000000e+00 1.3e+00 5.01e-01  5e-01  5e-01 0:31.0
termination on tolflatfitness=1 (Mon Jun 26 14:36:36 2023)
final/bestever f-value = 2.000000e+00 2.000000e+00
incumbent solution: [0.21166598 0.35487093 1.0789582  0.55334557 0.36967477 0.22953888
 0.35043476 0.43827922 ...]
std deviations: [0.50035949 0.50282757 0.49184076 0.49674491 0.50039135 0.4973353
 0.49727918 0.4956

In [6]:
result = solver.best_solution[0]
print("Parameters of the best solution :\n {solution}".format(solution=result))
print(f'Fitness value of the solution: {solver.best_solution[1].result.fbest}')

Parameters of the best solution :
 [1.66924729e-01 5.29901964e-01 2.24129515e-01 1.35470724e-01
 5.64023879e-01 7.95019137e-01 7.84866795e-01 3.46569511e-01
 1.17260920e-01 4.61132484e-01 5.34555976e-01 9.11113559e-02
 2.45931831e-01 2.45348032e-01 6.62142104e-02 1.84637589e-01
 6.87182681e-03 8.47004948e-01 5.33111109e-06 7.68228801e-01
 2.38973347e-01 7.39507873e-01 1.95184335e-01 2.62137491e-01
 1.46497508e-01 2.94297775e-01 3.53260310e-02 3.31760411e-01
 2.21913033e-01 8.12129012e-02 4.24844617e-02 1.37718821e-01
 4.18212431e-01 2.18796473e-02 9.06655461e-03 4.72776478e-02
 5.46234946e-01 4.61109312e-01 2.35651442e-01 1.10335254e-01]
Fitness value of the solution: 2


In [7]:
schedule = CMAToScheduleTranslator().translate(result, jobs, due_dates, durations, env, orders)
schedule.created_in = env
schedule.created_for = orders
schedule.created_by = solver

In [8]:
#TODO: current solution could be a result of either a) bug in schedule building (more likely) or b) lack of feasibility checks, since they shouldn't be necessary
visualize_schedule_demo(schedule, env, orders)

In [9]:
fitness_values = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {fitness_values[0]}\nTardiness: {fitness_values[1]}\nDeviation: {fitness_values[2]}\nIdle Time: {fitness_values[3]}\nProfit: {fitness_values[4]}')

AttributeError: 'NoneType' object has no attribute 'tasks'

In [ ]:
values = [9,2,3,4,5,1,1,2,4,5,6,7,6,2,2,3,4,5,5,6]
sorted = []
for i in range(len(values)):
    idx = values.index(min(values)) # workstation id
    sorted.append(values[idx])
    values[idx] = float('inf')
print(sorted)
print(values)

[1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 9]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]


In [ ]:
encoder = EncodeForGA()
values, durations, all_jobs = encoder.translate(env, orders)

     
crossover = 'two_points' #NOTE: available in PyGAD: 'two_points', 'single_point', 'uniform', 'scattered'
selection = 'rws' #NOTE: available in PyGAD: 'sss' (steady state selection', 'rws' (roulette wheel), 'sus' (stochastic universal selection), 'rank' (rank selection), 'random' (random selection), 'tournament' (tournament selection)
mutation = 'swap' #NOTE: available options: 'swap', 'inversion', 'scramble', 'adaptive
#objective = 'makespan' #NOTE: available options: 'makespan', 'idle_time'
population_size = 100
max_generations = 2000

solver = HybridGreedyAgentSolver(values, durations, all_jobs, env, orders)
solver.initialize(mutation, crossover, selection, max_generations, population_size)
solver.run()

Done
Done


In [ ]:
result = solver.get_best()
fitness = solver.get_best_fitness()

print("Parameters of the best solution : {solution}".format(solution=result))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(fitness)))

Parameters of the best solution : [9, 0, 9, 1, 9, 5, 9, 10, 5, 0, 5, 5, 5, 15, 5, 20, 0, 0, 9, 13, 9, 18, 9, 22, 5, 21, 5, 23, 5, 25, 5, 30, 4, 0, 4, 8, 4, 11, 4, 12, 1, 0, 1, 4, 4, 17, 9, 24, 6, 0, 1, 8, 4, 18, 9, 27, 4, 21, 4, 27, 4, 28, 4, 30, 2, 0, 9, 29, 9, 34, 9, 38, 0, 2, 1, 9, 1, 13, 5, 35]
Fitness value of the best solution = 42


In [ ]:
new_result = solver.re_run_agent()
print("Parameters of the best solution : {solution}".format(solution=new_result[0]))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(new_result[1])))

Done
Parameters of the best solution : [9, 0, 9, 1, 9, 5, 9, 10, 6, 0, 6, 2, 6, 6, 6, 8, 8, 0, 8, 3, 6, 11, 9, 13, 3, 0, 3, 4, 6, 13, 6, 15, 1, 0, 3, 8, 3, 17, 3, 21, 6, 17, 6, 22, 3, 25, 3, 29, 0, 0, 0, 1, 8, 11, 6, 24, 2, 0, 8, 20, 8, 28, 8, 30, 5, 0, 9, 15, 6, 27, 6, 29, 5, 2, 6, 31, 3, 33, 3, 37]
Fitness value of the best solution = 41


In [ ]:
result = new_result[0]
fitness = new_result[1]

In [ ]:
schedule = GAToScheduleTranslator().translate(result, all_jobs, env, orders)
schedule.created_in = env
schedule.created_for = orders
visualize_schedule_demo(schedule, env, orders)
fitness_values = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {fitness_values[0]}\nTardiness: {fitness_values[1]}\nDeviation: {fitness_values[2]}\nIdle Time: {fitness_values[3]}\nProfit: {fitness_values[4]}')

Fitness Values:
Makespan: 42
Tardiness: 0
Deviation: 709
Idle Time: 97
Profit: 5000


In [ ]:
import pickle

#solver.environment = None # make file smaller, since the schedule already knows the environment and the orders
#solver.orders = None
#schedule.created_by = solver
schedule.created_by = 'HybridAgentSolver'
schedule.evaluation_results = []
schedule.evaluation_results.append(('makespan', fitness_values[0]))
schedule.evaluation_results.append(('tardiness', fitness_values[1]))
schedule.evaluation_results.append(('deviation', fitness_values[2]))
schedule.evaluation_results.append(('idle_time', fitness_values[3]))
schedule.evaluation_results.append(('profit', fitness_values[4]))
save_file = True
if(save_file):
    with open(f'results/HybridSolverTesting.pickle', 'wb') as outfile:
        pickled_data = pickle.dump(schedule, outfile)